In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("google/fleurs",'th_th')

Reusing dataset fleurs (/root/.cache/huggingface/datasets/google___fleurs/th_th/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from pythaiasr import asr

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.8/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [42]:
import jiwer

In [4]:
from tqdm.auto import tqdm

In [5]:
test=dataset['test']['path'][0]

In [6]:
dataset['test']['audio'][0]

{'path': '18398348885358836952.wav',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.86102295e-06,  1.96099281e-05,  5.14388084e-05]),
 'sampling_rate': 16000}

In [7]:
model_name="airesearch/wav2vec2-large-xlsr-53-th"

In [8]:
asr(test)

2022-10-03 10:53:38.136427: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


'เนื่องจากนี้ไปทางเหนือเพื่อเยี่ยมชมบทแม่พระฟาริมาศารเจ้าซึ่งเป็นสถานที่ที่มีชื่อเสียงไปทั่วโลกในเรื่องการประจัก์ของพ่ะแม่มารี'

In [9]:
from transformers import AutoProcessor, AutoModelForCTC
import torchaudio
import numpy as np
import torch

In [24]:
import logging.config
logging.config.dictConfig({
    'version': 1,
    'disable_existing_loggers': True,
})

In [25]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [27]:
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name)

In [28]:
input_values=processor(dataset['test']['audio'][0]['array'], return_tensors="pt", padding=True)

In [29]:
input_values

{'input_values': tensor([[0.0011, 0.0011, 0.0011,  ..., 0.0006, 0.0047, 0.0105]])}

In [30]:
logits = model(input_values.input_values).logits

In [31]:
pred_ids = torch.argmax(logits, dim=-1)[0]

In [32]:
processor.decode(pred_ids)

'เนื่องจากนี้ ไป ทางเหนือ เพื่อ เยี่ยมชม บท แม่ พระ ฟาริมา ศารเจ้า   ซึ่ง เป็น สถานที่ ที่ มีชื่อเสียง ไป ทั่วโลก ใน เรื่อง การ ประจัก์ ของ พ่ะแม่มารี'

In [33]:
def array2text(array):
  input_values=processor(array, return_tensors="pt", padding=True)
  logits = model(input_values.input_values).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]
  return processor.decode(pred_ids)

In [34]:
array2text(dataset['test']['audio'][1]['array'])

'รถโดยสาร ออกจาก สถานี รถ ระหว่าง มอลฑล   ข้าม แม่น้ำ ตลอด ทั้งวัน   โดย ส่วนใหญ่ แล้ว ลด ที่ มุ่งหน้า ไป ทาง ตะวันออกและ จากา บุม แทง   จะ ออกจาก สถานี ระหว่าง เวลา   หก จุด สาม สูน นาฬิกา   และ เจ็ด จุด สามศูนย์ นาฬิกา'

In [35]:
text_all=[]
for i in tqdm(list(dataset['test']['audio'])):
  try:
    text_all.append(array2text(i['array']))
  except:
    text_all.append(None)

  0%|          | 0/1021 [00:00<?, ?it/s]

In [36]:
text_all[4]

'ระเบิด แบบ ฟิชชัน ทำงาน บน หลักการ ที่ว่าการ รวม นิวเคลียร์ ที่ มี โปตอน และ น ตอน จำนวนมาก ต้อง ใช้ พลังงาน'

In [39]:
text_g_all = [i.replace(' ','') for i in dataset['test']['transcription']]

In [40]:
text_all_cal = [i.replace(' ','') for i in text_all]

In [43]:
jiwer.cer(text_g_all,text_all_cal)

0.12120503118268292